In [41]:
from qiskit import pulse, circuit, transpile, assemble, schedule, IBMQ, QuantumCircuit, quantum_info as qi 
import qiskit
from qiskit.visualization import plot_gate_map, plot_error_map
from qiskit.visualization.pulse_v2 import draw
from qiskit.pulse import ShiftPhase, GaussianSquare, Drag, ControlChannel, MeasureChannel, DriveChannel, Play, Schedule
import numpy as np
from qiskit.tools import job_monitor
from copy import deepcopy
import scipy.optimize as opt
import matplotlib.pyplot as plt

In [42]:
account_provider = IBMQ.load_account()
provider_opts = {'hub': 'ibm-q-ncsu', 'group': 'nc-state', 'project':'noiseanderror'}

# target backend
backend_name = 'ibmq_jakarta'

provider = IBMQ.get_provider(**provider_opts)
backend = provider.get_backend(backend_name)

# backend information
config = backend.configuration()
defaults = backend.defaults()
properties = backend.properties()

# instruction schedule map
inst_map = defaults.instruction_schedule_map

ibmqfactory.load_account:WARNING:2021-11-21 11:40:25,156: Credentials are already in use. The existing account in the session will be replaced.


In [43]:
qubit = 0
freq01_estimate = backend.defaults().qubit_freq_est[qubit]
frequencies = np.linspace(freq01_estimate -15e6, freq01_estimate + 15e6, 51)

In [44]:
print(frequencies)

[5.22135679e+09 5.22195679e+09 5.22255679e+09 5.22315679e+09
 5.22375679e+09 5.22435679e+09 5.22495679e+09 5.22555679e+09
 5.22615679e+09 5.22675679e+09 5.22735679e+09 5.22795679e+09
 5.22855679e+09 5.22915679e+09 5.22975679e+09 5.23035679e+09
 5.23095679e+09 5.23155679e+09 5.23215679e+09 5.23275679e+09
 5.23335679e+09 5.23395679e+09 5.23455679e+09 5.23515679e+09
 5.23575679e+09 5.23635679e+09 5.23695679e+09 5.23755679e+09
 5.23815679e+09 5.23875679e+09 5.23935679e+09 5.23995679e+09
 5.24055679e+09 5.24115679e+09 5.24175679e+09 5.24235679e+09
 5.24295679e+09 5.24355679e+09 5.24415679e+09 5.24475679e+09
 5.24535679e+09 5.24595679e+09 5.24655679e+09 5.24715679e+09
 5.24775679e+09 5.24835679e+09 5.24895679e+09 5.24955679e+09
 5.25015679e+09 5.25075679e+09 5.25135679e+09]


In [45]:
def spectroscopy_scheds(qubit, pulse_params, backend):
    freq_param = circuit.Parameter("frequency")
    with pulse.build(backend=backend, name="spectroscopy") as schedule:
        pulse.shift_frequency(freq_param, DriveChannel(qubit))
        pulse.play(
            GaussianSquare(duration=pulse_params["duration"], 
                           amp = pulse_params["amp"], 
                           sigma=pulse_params["sigma"], 
                           width=pulse_params["width"]), DriveChannel(qubit))
        pulse.shift_frequency(-freq_param, DriveChannel(qubit))
        
    return schedule, freq_param

def template_circuit(freq_param):
    temp_circ = QuantumCircuit(1)
    temp_circ.append(circuit.Gate(name="Spec", num_qubits=1, params=[freq_param]), (0,))
    temp_circ.measure_active()
    return temp_circ

def spectroscopy_circuits(qubit, frequencies, pulse_params, backend):
    sched, freq_param = spectroscopy_scheds(qubit, pulse_params, backend)
    circuit = template_circuit(freq_param)
    circuit.add_calibration("Spec", (qubit,), sched, params=[freq_param])
    
    center_freq = backend.defaults().qubit_freq_est[qubit]
    
    circs = []
    for freq in frequencies:
        freq_shift = freq
        freq_shift -= center_freq
        freq_shift = np.round(freq_shift, decimals=3)
        assigned_circ = circuit.assign_parameters({freq_param: freq_shift}, inplace=False)
        circs.append(assigned_circ)
    return circs

In [46]:
pulse_params = {}

pulse_params["duration"] = 1024
pulse_params["amp"] = 0.1
pulse_params["sigma"] = 256
pulse_params["width"] = 0
schedules = spectroscopy_scheds(0, pulse_params, backend)

In [47]:
circs = spectroscopy_circuits(qubit, frequencies, pulse_params, backend)

In [48]:
transpiled_circs = transpile(circs, backend, initial_layout=[0])

In [49]:
transpiled_circs[0].draw()

┌─────────────────────────┐ ░ ┌─┐
      q_0 -> 0 ┤ Spec(-15000000.0000000) ├─░─┤M├
               └─────────────────────────┘ ░ └╥┘
ancilla_0 -> 1 ───────────────────────────────╫─
                                              ║ 
ancilla_1 -> 2 ───────────────────────────────╫─
                                              ║ 
ancilla_2 -> 3 ───────────────────────────────╫─
                                              ║ 
ancilla_3 -> 4 ───────────────────────────────╫─
                                              ║ 
ancilla_4 -> 5 ───────────────────────────────╫─
                                              ║ 
ancilla_5 -> 6 ───────────────────────────────╫─
                                              ║ 
    measure: 1/═══════════════════════════════╩═
                                              0

In [50]:
job = backend.run(transpiled_circs, job_name="spectroscopy", shots=2048)
job_id = job.job_id()
print(job.job_id())

619a768168476d6e298a8394


In [ ]:
# 11/21/spectroscopy
# jakarta (11:51) 619a768168476d6e298a8394
results = backend.retrieve_job(job_id).result()